<a href="https://colab.research.google.com/github/alexvishnevskiy/PetFinder/blob/master/Breed_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#try petpy

In [1]:
!pip install torch pytorch-lightning timm python-box -U albumentations wandb torch-optimizer > /dev/null

tcmalloc: large alloc 1147494400 bytes == 0x562a44208000 @  0x7f5129162615 0x562a0bba54cc 0x562a0bc8547a 0x562a0bba82ed 0x562a0bc99e1d 0x562a0bc1be99 0x562a0bc169ee 0x562a0bba9bda 0x562a0bc1bd00 0x562a0bc169ee 0x562a0bba9bda 0x562a0bc18737 0x562a0bc9ac66 0x562a0bc17daf 0x562a0bc9ac66 0x562a0bc17daf 0x562a0bc9ac66 0x562a0bc17daf 0x562a0bbaa039 0x562a0bbed409 0x562a0bba8c52 0x562a0bc1bc25 0x562a0bc169ee 0x562a0bba9bda 0x562a0bc18737 0x562a0bc169ee 0x562a0bba9bda 0x562a0bc17915 0x562a0bba9afa 0x562a0bc17c0d 0x562a0bc169ee


In [2]:
!unzip /content/drive/MyDrive/PetFinder/breeds/cat_breeds.zip -d cat_breeds > /dev/null
!unzip /content/drive/MyDrive/PetFinder/breeds/dog_breeds.zip -d dog_breeds > /dev/null
!unzip /content/drive/MyDrive/PetFinder/breeds/dog-breed-identification.zip -d dog_breeds_2 > /dev/null

!mkdir cat_breeds_2
!tar -xvf /content/drive/MyDrive/PetFinder/breeds/annotations.tar.gz --directory cat_breeds_2 > /dev/null
!tar -xvf /content/drive/MyDrive/PetFinder/breeds/images.tar.gz --directory cat_breeds_2 > /dev/null

!mkdir PetFinder_old
!unzip /content/drive/MyDrive/PetFinder/breeds/PetFinder_old/train.csv.zip -d PetFinder_old > /dev/null
!unzip /content/drive/MyDrive/PetFinder/breeds/PetFinder_old/train_images.zip -d PetFinder_old > /dev/null
!cp /content/drive/MyDrive/PetFinder/breeds/PetFinder_old/breed_labels.csv -d PetFinder_old

In [3]:
import pandas as pd
import numpy as np
from glob import glob
from pathlib import Path
from tqdm import tqdm
from box import Box
import wandb
import cv2
import os

from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from albumentations.pytorch.transforms import ToTensorV2
from torchmetrics import ConfusionMatrix, Accuracy, F1, Precision, Recall
from sklearn.metrics import classification_report
import albumentations as A

from pytorch_lightning import LightningDataModule, LightningModule
from pytorch_lightning.utilities.seed import seed_everything
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning import callbacks
import pytorch_lightning as pl

import torch
from timm import create_model
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
#import torch_optimizer as optim

In [ ]:
#добавить данные с предыдущей соревы
#torch.utils.data.WeightedRandomSampler

In [4]:
class paths:
  #cat_breeds_petfinder = Path('/content/cat_breeds/')
  cat_breeds_oxford = Path('/content/cat_breeds_2/')
  dog_breeds = Path('/content/dog_breeds/')
  dog_breeds_kaggle = Path('/content/dog_breeds_2/')
  petfinder_old = Path('/content/PetFinder_old/')

#out columns
columns = ['filepath', 'breed', 'data_source']

In [5]:
seed_everything(2021, workers=True)

Global seed set to 2021


2021

### Data

In [6]:
class DogsCatsData:
  def __init__(self, paths, columns):
    self.paths = paths
    self.columns = columns

  def get_data(self, min_th = 10, keep_th = 300):
    data = []
    for attr in vars(self.paths):
      if not attr.startswith('_'):
        data_function = getattr(self, f'{attr}_data')
        path = getattr(paths, attr)
        data.append(data_function(path))

    data = pd.concat([d[self.columns] for d in data])
    data = self._preprocess_data(data, min_th, keep_th)
    data = self._label_data(data)
    return data

  def _preprocess_data(self, data, min_th = 15, keep_th = 300):
    data['breed'] = data['breed'].str.lower()
    data['breed'] = data['breed'].apply(lambda x: x.replace('_', ' '))
    data = self._class_corections(data)

    breeds_sizes = data.groupby('breed').size()
    valid_breeds = breeds_sizes[breeds_sizes > min_th].index

    data = (
        data[data.breed.isin(valid_breeds)]
        .groupby('breed')
        .apply(lambda x: x.sample(keep_th, replace = True))
        .reset_index(drop=True)
        .drop_duplicates()
        )
    return data

  def _label_data(self, data):
    self.le = LabelEncoder()
    data['breed_id'] = self.le.fit_transform(data['breed'].values)
    return data

  def _class_corections(self, data):
    class_corection = {
        'afghan': 'afghan hound',
        'airedale': 'airedale terrier',
        'blenheim': 'blenheim spaniel',
        'boston bull': 'boston terrier',
        'chinese crested dog': 'chinese crested',
        'chow chow': 'chow',
        'cocker': 'cocker spaniel',
        'dalmation': 'dalmatian',
        'doberman pinscher': 'doberman',
        'english springer': 'english springer spaniel',
        'german sheperd': 'german shepherd',
        'german shepherd dog': 'german shepherd',
        'german shorthaired': 'german short-haired pointer',
        'irish spaniel': 'irish water spaniel',
        'jack russell terrier (parson russell terrier)': 'jack russell terrier',
        'labrador retriever': 'labrador',
        'leonberg': 'leonberger',
        'lhasa': 'lhasa apso',
        'maltese': 'maltese dog',
        'mex hairless': 'mexican hairless',
        'pekinese': 'pekingese',
        'pit bull': 'pit bull terrier',
        'rhodesian': 'rhodesian ridgeback',
        'scottish terrier scottie': 'scottish terrier',
        'shetland sheepdog sheltie': 'shetland sheepdog',
        'shih tzu': 'shih-tzu',
        'sphynx (hairless cat)': 'sphynx',
        'staffordshire bull terrier': 'staffordshire bullterrier',
        'west highland white terrier westie': 'west highland white terrier',
        'wire-haired fox terrier': 'wirehaired terrier',
        'yorkie': 'yorkshire terrier', 
        'yorkshire terrier yorkie': 'yorkshire terrier',
        }
    
    for breed in data['breed'].unique():
      if breed not in class_corection.keys():
        class_corection[breed] = breed

    data['breed'] = data['breed'].map(class_corection)
    return data

  @staticmethod
  def get_label_weights(data, device):
    label_count = (
        data['breed_id']
        .value_counts()
        .to_frame()
        .sort_index()
        .values
    )
    weigths = torch.from_numpy(np.power(label_count, -1.)).float().squeeze()
    return weigths.to(device)

  @staticmethod
  def cat_breeds_oxford_data(path: Path):
    with open(path/'annotations/list.txt', 'r') as f:
      for _ in range(6):
        f.readline()
      cats_oxford = pd.read_csv(f, sep=" ", header=None)

      cats_oxford.columns = ["id", "CLASS-ID", "SPECIES", "BREED-ID"]
      cats_oxford['breed'] = cats_oxford['id'].apply(lambda x: ' '.join(x.split('_')[:-1]))
      cats_oxford['data_source'] = 'cats_oxford'
      cats_oxford['filepath'] = cats_oxford['id'].apply(lambda x: path/f'images/{x}.jpg')
    return cats_oxford

  @staticmethod
  def cat_breeds_petfinder_data(path: Path):
    cats = {'id': [], 'breed': [], 'filepath': []}
    for path in glob(os.path.join(path, 'images/*/*.jpg')):
      breed, id = path.split('/')[-2:]
      id = id.rstrip('.jpg')
      
      cats_petfinder['id'].append(id)
      cats_petfinder['breed'].append(breed)
      cats_petfinder['filepath'].append(path)
      
      cats_petfinder = pd.DataFrame(cats)
      cats_petfinder['data_source'] = 'cats_petfinder'
    return cats_petfinder

  @staticmethod
  def dog_breeds_data(path: Path):
    dogs = pd.read_csv(path/'dogs.csv')
    dogs['data_source'] = 'dog_breeds'
    dogs['filepath'] = dogs['filepaths'].apply(lambda x: path/x)
    dogs.rename(columns = {'labels': 'breed'}, inplace = True)
    return dogs

  @staticmethod
  def dog_breeds_kaggle_data(path: Path):
    dogs_kaggle = pd.read_csv(path/'labels.csv')
    dogs_kaggle['filepath'] = dogs_kaggle['id'].apply(lambda x: path/f'train/{x}.jpg')
    dogs_kaggle['data_source'] = 'dogs_kaggle'
    return dogs_kaggle

  @staticmethod
  def petfinder_old_data(path: Path):
    train_petfinder = pd.read_csv(path/'train.csv')
    mappings = pd.read_csv(path/'breed_labels.csv')
    mappings = (
        mappings[['BreedID', 'BreedName']]
        .set_index('BreedID')
        .to_dict()['BreedName']
    )

    train_petfinder['breed'] = train_petfinder['Breed1'].map(mappings)
    train_petfinder['filepath'] = train_petfinder['PetID'].apply(lambda x: path/f'{x}-1.jpg')
    train_petfinder['data_source'] = 'petfinder_old'

    most_common = train_petfinder['breed'].value_counts().index[0]
    train_petfinder['breed'] = train_petfinder['breed'].fillna(most_common)
    return train_petfinder

### Dataset

In [7]:
class BreedDataset(Dataset):
  def __init__(self, df, img_size = (224, 224), transforms = None):
    self.df = self._make_dataset(df)
    self.img_size = img_size
    self.transforms = self.__transforms(transforms)

  def _make_dataset(self, df):
    print('dropping wrong images...')
    for i, row in df.iterrows():
      img = self.read_img(row.filepath)
      if not isinstance(img, np.ndarray):
        df.drop(axis = 0, index = i, inplace = True)
    return df

  def __transforms(self, transforms):
    if transforms is None:
      transforms = A.Compose([
                       A.Resize(*self.img_size),
                       A.Normalize(
                           mean = [0.485, 0.456, 0.406],
                           std = [0.229, 0.224, 0.225],
                           always_apply = True
                           ),
                       ToTensorV2(),
                       ])
    return transforms

  def __len__(self):
    return len(self.df)

  def __getitem__(self, indx):
    path = self.df.iloc[indx].filepath
    label = self.df.iloc[indx].breed_id
    img = self.prepare_img(path)
    return img, label

  @staticmethod
  def read_img(path):
    if isinstance(path, Path):
      path = path.as_posix()
    img = cv2.imread(path)
    return img

  def prepare_img(self, path):
    img = self.read_img(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = self.transforms(image=img)['image']
    return img

In [8]:
class CustomDataModule(LightningDataModule):
  def __init__(
      self,
      data: DogsCatsData,
      transforms: A.Compose = None,
      weights = None,
      test_size = 0.25,
      img_size = (224, 224),
      batch_size = 64
      ):
    super().__init__()
    self.train_df, self.val_df = self.split_data(data, test_size)
    self.batch_size = batch_size
    self.transforms = transforms
    self.img_size = img_size
    self.weights = weights

  @staticmethod
  def split_data(data, test_size):
    train_df, val_df = train_test_split(
        data,
        test_size = test_size,
        random_state = 2021
        )
    return train_df, val_df

  def train_dataloader(self):
    train_split = BreedDataset(
        self.train_df, 
        self.img_size, 
        self.transforms
        )
    return DataLoader(
        train_split,
        batch_size=self.batch_size, 
        shuffle=True, 
        num_workers=4,
        )

  def val_dataloader(self):
    val_split = BreedDataset(self.val_df, self.img_size)
    return DataLoader(
        val_split, 
        batch_size=self.batch_size, 
        shuffle=False,
        num_workers=4,
        )

### Model

In [9]:
class CustomModel(LightningModule):
  def __init__(self, cfg):
    super().__init__()
    self.cfg = cfg
    self.__build_model()
    self.save_hyperparameters(cfg)

  def __build_model(self):
    self.backbone = create_model(
        self.cfg.model_name, 
        drop_rate = self.cfg.dropout_backbone, 
        pretrained=True, 
        num_classes=0, 
        in_chans=3
        )
    self.fc = nn.Sequential(
        nn.Dropout(self.cfg.dropout_fc),
        nn.LazyLinear(self.cfg.num_classes)
        )
    
  def forward(self, x):
    f = self.backbone(x)
    out = self.fc(f)
    return out

  def configure_optimizers(self):
    optimizer = eval(self.cfg.optimizer.name)(
        self.parameters(), 
        **self.cfg.optimizer.params
        )
    scheduler = eval(self.cfg.scheduler.name)(
        optimizer,
        **self.cfg.scheduler.params
        )
    return [optimizer], [scheduler]

  def __share_step(self, batch):
    img, labels = batch
    logits = self(img)
    preds = logits.argmax(dim = -1)

    loss = F.cross_entropy(logits, labels, weight = self.cfg.weights)
    return loss, labels, preds

  def __share_epoch(self, outputs, stage):
    def calculate_metrics(preds, labels):
      accuracy_score = Accuracy(num_classes = self.cfg.num_classes, average = 'macro')
      f1_score = F1(self.cfg.num_classes, average = 'macro')
      pr_score = Precision(self.cfg.num_classes, average = 'macro')
      r_score = Recall(self.cfg.num_classes, average = 'macro')

      accuracy = accuracy_score(preds, labels)
      f1 = f1_score(preds, labels)
      precision = pr_score(preds, labels)
      recall = r_score(preds, labels)
      return {
          'accuracy': accuracy, 
          'f1': f1, 
          'precision': precision, 
          'recall': recall
          }

    preds = torch.cat([out['preds'] for out in outputs]).cpu()
    labels = torch.cat([out['labels'] for out in outputs]).cpu()

    metrics = calculate_metrics(preds, labels)
    for k, v in metrics.items():
      self.log(f'{stage}_{k}', v)

  def training_step(self, batch, batch_idx):
    loss, labels, preds = self.__share_step(batch)
    self.log('train_loss', loss)
    return {'loss': loss, 'preds': preds, 'labels': labels}
        
  def validation_step(self, batch, batch_idx):
    loss, labels, preds = self.__share_step(batch)
    self.log('val_loss', loss)
    return {'loss': loss, 'preds': preds, 'labels': labels}

  def training_epoch_end(self, outputs):
    self.__share_epoch(outputs, 'train')

  def validation_epoch_end(self, outputs):
    self.__share_epoch(outputs, 'val')

  def predict_step(self, batch, batch_idx, dataloader_idx=0):
    img, labels = batch
    return self(img)

In [10]:
class ImagePredictionLogger(callbacks.Callback):
    def __init__(self, val_samples):
        super().__init__()
        self.val_imgs, self.val_labels = val_samples

    def on_validation_epoch_end(self, trainer, pl_module):
        val_imgs = self.val_imgs.to(device=pl_module.device)
        val_labels = self.val_labels.to(device=pl_module.device)
        
        logits = pl_module(val_imgs)
        preds = torch.argmax(logits, -1)

        trainer.logger.experiment.log({
            "examples":[wandb.Image(x, caption=f"Pred:{pred}, Label:{y}") 
                           for x, pred, y in zip(val_imgs, 
                                                 preds, 
                                                 val_labels)]
            })

### Training

In [11]:
cfg = {
    'model_name': 'swin_large_patch4_window7_224',
    'dropout_backbone': 0,
    'dropout_fc': 0,
    'epoch': 10,
    'batch_size': 16,
    'img_size': (224, 224),
    'test_size': 0.2,
    'device': 'cuda:0',
    'weights': None,
    'optimizer':{
        'name': 'optim.AdamW',
        'params':{
            'lr': 5e-5,
            'weight_decay': 1e-4,#1e-3
        },
    },
    'scheduler':{
        'name': 'optim.lr_scheduler.CosineAnnealingWarmRestarts',
        'params':{
            'T_0': 10,
            'eta_min': 1e-6
        },
    },
    'logger': {
        'save_dir': '/content/drive/MyDrive/PetFinder/models',
        'name': 'swin_large_breed_more_data',
        'project': 'Breeds',
        'log_model': True,
    },
    'trainer': {
        'gpus': 1,
        'accumulate_grad_batches': 4,
        'auto_lr_find': False,
        'progress_bar_refresh_rate': 3,
        'fast_dev_run': False,
        'num_sanity_val_steps': 2,
        #'overfit_batches': 1,
        'resume_from_checkpoint': None,
    },
}
cfg = Box(cfg)

In [12]:
train_transforms = A.Compose([
                 A.HorizontalFlip(p = 0.5),
                 A.VerticalFlip(p = 0.5),
                 A.RandomBrightnessContrast(p=0.3),
                 A.ShiftScaleRotate(p=0.3),
                 #A.RandomCrop(width=cfg.img_size[0], height=cfg.img_size[1]),
                 A.RandomResizedCrop(*cfg.img_size, scale = (0.7, 1)),
                 A.Resize(height=cfg.img_size[0], width=cfg.img_size[1]),
                 A.Normalize(
                     mean = [0.485, 0.456, 0.406],
                     std = [0.229, 0.224, 0.225],
                     always_apply = True
                     ),
                 ToTensorV2(),                                
                 ])

In [13]:
data = DogsCatsData(paths, columns).get_data()
cfg.num_classes = data['breed'].nunique()
cfg.train_transforms = train_transforms
#cfg.weights = DogsCatsData.get_label_weights(data, cfg.device)

model = CustomModel(cfg)
datamodule = CustomDataModule(
    data, 
    cfg.train_transforms, 
    cfg.weights,
    cfg.test_size, 
    cfg.img_size,
    cfg.batch_size
    )

val_samples = next(iter(datamodule.val_dataloader()))
img_predictions = ImagePredictionLogger(val_samples)
earystopping = EarlyStopping(monitor="val_loss", patience = 3)
lr_monitor = callbacks.LearningRateMonitor('step')

loss_checkpoint = callbacks.ModelCheckpoint(
    dirpath = os.path.join(cfg.logger.save_dir, cfg.logger.name),
    filename=cfg.logger.name,
    monitor="val_loss",
    save_top_k=1,
    mode="min",
    save_last=False,
    )
wandb_logger = WandbLogger(
    name = cfg.logger.name,
    project = cfg.logger.project,
    log_model = True,
    )

Downloading: "https://github.com/SwinTransformer/storage/releases/download/v1.0.0/swin_large_patch4_window7_224_22kto1k.pth" to /root/.cache/torch/hub/checkpoints/swin_large_patch4_window7_224_22kto1k.pth
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


dropping wrong images...


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [14]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
#more augmentaions
#freeze layers
#label smoothing

In [ ]:
trainer = pl.Trainer(
      max_epochs=cfg.epoch,
      logger = wandb_logger,
      callbacks=[
            lr_monitor, 
            loss_checkpoint, 
            earystopping,
            img_predictions,
            ],
      deterministic=True,
      **cfg.trainer,
      )
trainer.fit(model, datamodule=datamodule, ckpt_path="/content/drive/MyDrive/PetFinder/models/swin_large_breed_more_data/swin_large_breed_more_data-v2.ckpt")
wandb.finish()

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/callback_connector.py:91: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=3)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
  f"Setting `Trainer(progress_bar_refresh_rate={progress_bar_refresh_rate})` is deprecated in v1.5 and"
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
Restoring states from the checkpoint path at /content/drive/MyDrive/PetFinder/models/swin_large_breed_more_data/swin_large_breed_more_data-v2.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
wandb: Currently logged in as: sashnevskiy (use `wandb login --relogin` to force relogin)


/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/checkpoint_connector.py:248: UserWarning: You're resuming from a checkpoint that ended mid-epoch. Training will start from the beginning of the next epoch. This can cause unreliable results if further training is done, consider using an end of epoch checkpoint.
  "You're resuming from a checkpoint that ended mid-epoch."
Restored all states from the checkpoint file at /content/drive/MyDrive/PetFinder/models/swin_large_breed_more_data/swin_large_breed_more_data-v2.ckpt

  | Name     | Type            | Params
---------------------------------------------
0 | backbone | SwinTransformer | 194 M 
1 | fc       | Sequential      | 287 K 
---------------------------------------------
195 M     Trainable params
0         Non-trainable params
195 M     Total params
781.132   Total estimated model params size (MB)
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:617: UserWarning: Chec

Validation sanity check: 0it [00:00, ?it/s]

dropping wrong images...


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Global seed set to 2021
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


dropping wrong images...


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]